In [1]:
import rasterio
import math
import numpy as np
from rasterio.enums import Resampling
from scipy import ndimage, misc
# from rasterio.windows import Window
# import math
# import zarr

In [13]:
# vhm = zarr.open('../tmp/bhm.zarr')

In [2]:
# vhm.shape

In [3]:
# vhm.dtype

In [7]:
src = rasterio.open('tmp/0-VHM-sao-paulo-city.tif')

In [8]:
profile = src.profile

In [9]:
downscale = 2
upscale = 1
chunk_size = 20000
w_scaled = math.floor(src.width / upscale)
h_scaled = math.floor(src.height / upscale)
w_size = math.ceil(w_scaled/chunk_size)
h_size = math.ceil(h_scaled/chunk_size)

In [12]:
vhm = np.ones((src.count, int(src.height / downscale), int(src.width / downscale)), dtype='uint8')

In [15]:
# Gerando raster de 1m com 8bits

# resample data to target shape

data = src.read(
    out=vhm,
    # out_shape=(
    #     src.count,
    #     int(src.height / downscale),
    #     int(src.width / downscale)
    # ),
    resampling=Resampling.average
)

In [25]:
# scale image transform
transform = src.transform * src.transform.scale(downscale, downscale)

In [28]:
transform

Affine(1.0, 0.0, 313389.86,
       0.0, -1.0, 7416156.48)

In [26]:
vhm.shape, vhm.dtype

((1, 70330, 47227), dtype('uint8'))

In [30]:
src.height

140661

In [33]:
profile.update(
    dtype=rasterio.uint8,
    count=1,
    height=vhm.shape[1],
    width=vhm.shape[2],
    transform=transform)

with rasterio.open('resultados/0-VHM-sao-paulo-city-1m-8bits.tif', 'w', **profile) as dst:
    dst.write(vhm)

In [46]:
vhm_100m = ndimage.uniform_filter(vhm[0, :, :], size=(200, 200))

In [48]:
with rasterio.open('resultados/0-VHM-sao-paulo-city-1m-8bits-raio-100m.tif', 'w', **profile) as dst:
    dst.write(vhm_100m, 1)

In [49]:
vhm_1000m = ndimage.uniform_filter(vhm[0, :, :], size=(1000, 1000))

In [50]:
with rasterio.open('resultados/0-VHM-sao-paulo-city-1m-8bits-raio-1000m.tif', 'w', **profile) as dst:
    dst.write(vhm_1000m, 1)

In [5]:
w_scaled, h_scaled

(94455, 140661)

In [6]:
w_size, h_size

(5, 8)

In [7]:
vhm_temp = zarr.open(
    '../tmp/bhm.zarr', 
    mode='w', 
    shape=(w_size*chunk_size, h_size*chunk_size), 
    chunks=(chunk_size,chunk_size), 
    dtype=np.float32
)

In [8]:
vhm = np.ones((src.count, chunk_size, chunk_size), dtype='float32')

for w in range(w_size):
    for h in range(h_size):
        print(w,h)
        
        
        w1 = (w+1)*chunk_size*upscale
        h1 = (h+1)*chunk_size*upscale

        # if w1 > w_scaled:
        #     w1 = src.width

        # if h1 > h_scaled:
        #     h1 = src.height
      
        data = src.read(
            # out_shape=(src.count, chunk_size, chunk_size),
            out=vhm,
            window=Window.from_slices(
                slice(w*chunk_size*upscale, w1), 
                slice(h*chunk_size*upscale, h1)
            ),
            # resampling=Resampling.average,
            boundless=True,
            fill_value=0.
        )

        vhm_temp[w*chunk_size:w1//upscale, h*chunk_size:h1//upscale] = vhm[0]
            

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
